In [ ]:
def check_gpu():
    if tf.test.gpu_device_name() != '':
        print("GPU successfully connected.")
    else:
        print("Please connect a GPU.")

%pip install transformers datasets accelerate bitsandbytes pyarrow

from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
import torch
import tensorflow as tf

GPU = tf.config.list_physical_devices('GPU')
if GPU:
    print("GPU is available.")
else:
    print("GPU is not available.")

check_gpu()


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
GPU is available.
GPU successfully connected.


# **#Task 2**

Data Preprocessing : 1. Reformat the dataset into Llama 2 chat model template
<br>
                     2. Push the reformated data to Hugging Face Hub ( This dataset will be used further)

Following is the prompt template of Llama 2 chat model

![image.png](attachment:763412bc-6f1a-4587-8c62-31931f611a26.png)


In [ ]:
#Here goes the code for data preprocessing
!pip install -q datasets
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `test` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `test`


In [ ]:
from datasets import load_dataset
import re
dataset = load_dataset('jizzu/llama2_indian_law_v2')

dataset = dataset['train']

def transform_conversation(example):
    conversation_text = example['text']
    segments = conversation_text.split('###')

    reformatted_segments = []

    for i in range(1, len(segments) - 1, 2):
        human_text = segments[i].strip().replace('Human:', '').strip()

        if i + 1 < len(segments):
            assistant_text = segments[i+1].strip().replace('Assistant:', '').strip()

            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] {assistant_text} </s>')
        else:
            reformatted_segments.append(f'<s>[INST] {human_text} [/INST] </s>')

    return {'text': ''.join(reformatted_segments)}

transformed_dataset = dataset.map(transform_conversation)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


train.csv:   0%|          | 0.00/13.4M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/900k [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/112k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/24607 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/455 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/276 [00:00<?, ? examples/s]

Map:   0%|          | 0/24607 [00:00<?, ? examples/s]

In [ ]:
transformed_dataset.push_to_hub("newdata")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/25 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Subh2702/newdata/commit/8efd26feaa0414314865ab8ee3d0a9d4457a3afa', commit_message='Upload dataset', commit_description='', oid='8efd26feaa0414314865ab8ee3d0a9d4457a3afa', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/Subh2702/newdata', endpoint='https://huggingface.co', repo_type='dataset', repo_id='Subh2702/newdata'), pr_revision=None, pr_num=None)